## Notebook F: Dictionary Matching

#### Prerequisites:
- a simulated rawdata file containing time-varying magnetisation.
- a pre-computed dictionary of time-resolved magnetisation for many tissue parameter combinations.

#### Goals:
- reconstruct time-resolved images containing the MRF signal.

#### Content overview: 
- loading previously reconstructed data
- averaging the dictionary to the same temporal resolution as the reconstructions
- creating T1 and T2 maps from previously simulated data using dictionary matching 

In [ ]:
from pathlib import Path
import os 
import auxiliary_functions as aux

import numpy as np 
import sirf.Gadgetron as pMR
import matplotlib.pyplot as plt

# this is where we store the properly formatted data
root_path = aux.root_path
fpath_input = root_path / "Output"
fpath_output = fpath_input

In [ ]:
# we load the images we reconstructed before
fname_ad = fpath_input / "output_e_timeresolved_mrf_simulation.h5"
ad = pMR.AcquisitionData(str(fname_ad))

fname_simulated_recon = fpath_input / "output_e_timeresolved_recon_mrf_simulation.npy"
recon_arr = np.load(fname_simulated_recon)
recon = pMR.ImageData()
recon.from_acquisition_data(ad)
recon = recon.fill(recon_arr)

To perform the matching we have to load a pre-computed dictionary. Since we used a sliding window to reconstruct images at lower temporal resolution, but higher image quality the pre-computed dictionary must be brought to the same temporal resolution. This can be done with an auxiliary function does this based on the AcquisitionData we previously used for the images.

In [ ]:
# 
fname_dict = Path("/media/sf_CCPPETMR/TestData/Input/xDynamicSimulation/pDynamicSimulation/Fingerprints/dict_70_1500.npz")
mrfdict = np.load(fname_dict)

dict_theta = mrfdict['dict_theta']

dict_mrf = mrfdict['dict_norm']
dict_mrf = np.transpose( aux.apply_databased_sliding_window(ad, np.transpose(dict_mrf)))

dict_us_factor = 10
dict_mrf = dict_mrf[0:-1:dict_us_factor,:]
dict_theta = dict_theta[0:-1:dict_us_factor,:]

We have to convert the image in the shape `(#time points, #pixels)`

In [ ]:
print("Our dictionary to match is of size {}".format(dict_mrf.shape))
img_series = recon.as_array()
img_shape = img_series.shape[1:]
img_series_1d = np.transpose(np.reshape(img_series,(img_series.shape[0], -1)))

In [ ]:
# this checks the largest overlap between time-profile and dictionary entries
# if the RAM overflows this will catch it and perform the task in multiple sets.
dict_match = aux.match_dict(dict_mrf, dict_theta, img_series_1d)
dict_match = np.reshape(dict_match, (*img_shape, -1))

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

f,ax = plt.subplots(1,2)

divider = make_axes_locatable(ax[0])
cax = divider.append_axes('right', size='5%', pad=0.05)

im = ax[0].imshow(np.abs(dict_match[:,:,1])/1000,cmap='jet',vmin=0,vmax=2.5)
cbar = f.colorbar(im, cax=cax, ticks=[0, 0.8, 1.6, 2.4])
cbar.ax.tick_params(labelsize=12)

ax[0].axis("off")
ax[0].set_title("T1 (s)")


divider = make_axes_locatable(ax[1])
cax = divider.append_axes('right', size='5%', pad=0.05)

im = ax[1].imshow(np.abs(dict_match[:,:,2]),cmap='magma',vmin=0,vmax=150)
cbar = f.colorbar(im, cax=cax, ticks=[0, 50, 100, 150])
cbar.ax.tick_params(labelsize=12)

ax[1].axis("off")
ax[1].set_title("T2 (ms)")

fig_path = root_path / "Figures"
fig_path.mkdir(exist_ok=True)

fname_out = fig_path / "fig_f_dictionary_matching_static.png"
plt.savefig(str(fname_out), dpi=300)
plt.show()

In [ ]:
import nibabel as nib
T1GT = nib.load(str(fpath_input/ "output_c_static_ground_truth_T1_ms.nii"))
T2GT = nib.load(str(fpath_input/ "output_c_static_ground_truth_T2_ms.nii"))

f,ax = plt.subplots(2,2)

ax[0,0].imshow(np.abs(dict_match[:,:,1]),cmap='jet',vmin=0,vmax=2500)
ax[0,0].axis("off")
ax[0,0].set_title("T1")

divider = make_axes_locatable(ax[0,1])
cax = divider.append_axes('right', size='5%', pad=0.05)

im = ax[0,1].imshow(np.transpose(np.abs(np.squeeze(T1GT.get_fdata()))),cmap='jet',vmin=0,vmax=2500)
cbar = f.colorbar(im, cax=cax, ticks=[0, 800, 1600, 2400])
cbar.ax.tick_params(labelsize=12)
cbar.set_label(f'T1(ms)')

ax[0,1].axis("off")
ax[0,1].set_title("T1 GT")

ax[1,0].imshow(np.abs(dict_match[:,:,2]),cmap='magma',vmin=0,vmax=150)
ax[1,0].axis("off")
ax[1,0].set_title("T2")


divider = make_axes_locatable(ax[1,1])
cax = divider.append_axes('right', size='5%', pad=0.05)

im = ax[1,1].imshow(np.transpose(np.abs(np.squeeze(T2GT.get_fdata()))),cmap='magma',vmin=0,vmax=150)
cbar = f.colorbar(im, cax=cax, ticks=[0, 50, 100, 150])
cbar.ax.tick_params(labelsize=12)
cbar.set_label(f'T2(ms)')


ax[1,1].axis("off")
ax[1,1].set_title("T2 GT")

fig_path = root_path / "Figures"
fig_path.mkdir(exist_ok=True)

fname_out = fig_path / "fig_f_dictionary_matching_static_comparison.png"
plt.savefig(str(fname_out), dpi=300)
plt.show()

In [ ]:

img = nib.Nifti1Image(np.abs(dict_match[...,1]), np.eye(4))
fname_output = fpath_output / "output_f_fit_T1.nii"
nib.save(img,fname_output)

img = nib.Nifti1Image(np.abs(dict_match[...,2]), np.eye(4))
fname_output = fpath_output / "output_f_fit_T2.nii"
nib.save(img,fname_output)

### Recap
In this notebook we 
- averaged a high-resolution dictionary to the temporal resolution we reconstructed.
- performed dictionary matching to compute T1 and T2 maps.
- compared results of dictionary matching with ground truth T1 and T2 maps.

_Up next: combining motion and time-dependent magnetisation._